In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import *
import sklearn
from sklearn import metrics
from sklearn import tree
from sklearn import ensemble
from sklearn import preprocessing
sns.set()

## Data Import 

In [ ]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
col = train.columns

In [ ]:
#Data information
train.info()

In [ ]:
f=128903 #bullet styling
col # Colum names
for i, n in enumerate(col):
    print("{:c} index: {}  column name: {}".format(f,i,n))

## Preprocessing:

In [ ]:
def toDoPrint(List,f):
    str =""
    temp = ""
    for i in List:
        temp ="{:c} {}".format(f,i)
        print(temp)
        str += temp + "\n"
    return str
        
        
        
Todo = {"Missing data: Age, Cabin, Embarked",
        "Handling Data","Convert string values to numeric", 
        "Remove cols that do not contribute: Name, Cabin, Ticket and ?"
    }

todo_list = toDoPrint(Todo,f)

In [ ]:
def sexRatio(data):
    """m = male 
       f = female
       t = Total
       return an array [men%, female%, Total]
    """
    m=data.Sex[train.Sex=="male"].count()
    f=data.Sex[train.Sex=="female"].count()
    t=m+f
    return [m/t*100,f/t*100,t]

In [ ]:
def sumList(a,b):
    """return corresponding sum of each elements"""
    length = len(a)
    if(length != len(b)):
        print("A and B are not the same length")
        return []
    
    sum_v =[]
    for i in range(0,length):
        sum_v.append(a[i]+b[i])
        
    return sum_v
    
    
def pieChart(data, data1=[], both=False):
  
    
    if(both):
        ratio = sumList(sexRatio(data),sexRatio(data1))
    else:
        ratio = sexRatio(data)
    
    labels = 'Male', 'Female'
    sizes = [ratio[0], ratio[1]]
    

    fig1, ax1 = plt.subplots()
    ax1.pie(sizes,labels=labels, autopct='%1.1f%%',
            shadow=True, startangle=90)
    

    plt.show()

In [ ]:
pieChart(train,test,both=True)

## Missing data:

In [ ]:
def numerOfMissingValues(p, col, get_data = False):
    """numerOfMissingValues(pd, col, get_data = False)
       p: DataFrame
       col: column that needs to find the missing values

       Returns the total number of missing data
       return the pandas DatFrame of values if get_data is set to True
    """
    data = p[pd.isnull(p[col])]
    if get_data:
        return data
    else: 
        return len(data)

In [ ]:
# As shown on info there are 891 rows of data

missing =['Age', 'Cabin','Embarked']
for i in missing:
    print("{:c} Number of values missing in {} is: {}".format(f,i,numerOfMissingValues(train,i)))
    

***As shown above there are 177 missing values in Age, Cabin and Embarked**

### Handling Missing Data

In [ ]:
#Age: interpolate the missing data
train['Age'] = train['Age'].interpolate()
#Drop 'Name','Ticket','Cabin' because it is nor relavant to the model
cols = ['Name','Ticket','Cabin']
train = train.drop(cols,axis=1)

In [ ]:
dummies = []
cols = ['Pclass','Sex','Embarked']
for col in cols:
    dummies.append(pd.get_dummies(df[col]))


In [ ]:
titanic_dummies = pd.concat(dummies, axis=1)
df = train
df = pd.concat((df,titanic_dummies),axis=1)


In [ ]:
df = df.drop(['Pclass','Sex','Embarked'],axis=1)

### Converting String literals to Numeric:

In [ ]:
def stringToNum(row):
    if(row.Sex)=="female":
        row.Sex = 1
    else:
        row.Sex = 2

    if(row.Embarked)=="S":
        row.Embarked = 1
    elif row.Embarked == "C":
        row.Embarked = 2
    else:
        row.Embarked = 3
    return row

train =train.apply(stringToNum, axis='columns')
train.head(10)


### Classification:

In [ ]:
#df = train
x = df.values
y = df['Survived'].values
x = np.delete(x,1,axis=1)

In [ ]:
X_train,X_test,Y_train,Y_test = sklearn.model_selection.train_test_split(x,y,test_size=.3,shuffle=True)

In [ ]:
clf = tree.DecisionTreeClassifier(max_depth=5)
clf.fit(X_train,Y_train)
clf.score(X_test,Y_test)

In [ ]:
clf = ensemble.RandomForestClassifier(n_estimators=100)
clf.fit (X_train, Y_train)
clf.score (X_test, Y_test)

In [ ]:
clf = ensemble.GradientBoostingClassifier()
clf.fit (X_train, Y_train)
clf.score (X_test, Y_test)

In [ ]:
from sklearn.naive_bayes import GaussianNB
clf = GaussianNB()
clf.fit(x, y)
clf.score (X_test, Y_test)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

neigh = KNeighborsClassifier(n_neighbors=3)
neigh.fit(x, y)
neigh.score (X_test, Y_test)


In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.datasets import make_classification

clf = MLPClassifier(random_state=2, max_iter=30000).fit(X_train, Y_train)
clf.score(X_test, Y_test)